In [1]:
import pandas as pd

import os

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import json

In [2]:
directory_in = './file/2.url'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'crawler_1-20241205_203348.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：522



,No,Item Number,Url
0,1,232315264904,https://www.ebay.com/itm/232315264904?_ul=US&_...
1,2,335513085831,https://www.ebay.com/itm/335513085831?_ul=US&_...
2,3,255628697516,https://www.ebay.com/itm/255628697516?_ul=US&_...
3,4,235461387733,https://www.ebay.com/itm/235461387733?_ul=US&_...
4,5,334707355204,https://www.ebay.com/itm/334707355204?_ul=US&_...
...,...,...,...
517,518,256600835779,https://www.ebay.com/itm/256600835779?_ul=US&_...
518,519,256681642957,https://www.ebay.com/itm/256681642957?_ul=US&_...
519,520,335625681208,https://www.ebay.com/itm/335625681208?_ul=US&_...
520,521,334712574189,https://www.ebay.com/itm/334712574189?_ul=US&_...


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                             1
 Item Number                                         232315264904
 Url            https://www.ebay.com/itm/232315264904?_ul=US&_...
 Name: 0, dtype: object)

In [5]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp.status_code

1


200

In [6]:
soup = BeautifulSoup(resp.text, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="//thumbs.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//vi.vipr.ebaydesc.com" rel="dns-prefetch"/>
  <link href="//p.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//thumbs.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//q.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//pics.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//srx.main.ebayrtm.com" rel="dns-prefetch"/>
  <link href="//reco.ebay.com" rel="dns-prefetch"/>
  <link href="//ir.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//secureir.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//i.ebayimg.com" rel="dns-prefetch"/>
  <link href="//rover.ebay.com" rel="dns-prefetch"/>
  <link href="//i.ebayimg.cn" rel="dns-prefetch"/>
  <link href="//ir.ebaystatic.cn" rel="dns-prefetch"/>
  <link crossorigin="" href="//i.ebayimg.cn" rel="preconnect"/>
  <link crossorigin="" href="//ir.ebaystatic.cn" rel="preconnect"/>
  <script>
   $ssgST=new Date().getTime(

In [7]:
dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

dict_

{'o': {'l': 1,
  'g': {'VINODE_FULL_HEIGHT': {'enableFullHeight': False},
   'enableRumWeb': True,
   'permanentSignal': True,
   'enableATCRedesign': 'false',
   'lazyLoaderType': 'NATIVE',
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'enableRumWeb': {'variant': '276298',
      'value': True},
     'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'}, 'dropTag': 'server'},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'false',
       'DELI_ENABLED': 'true'},
      'dropTag': 'server'},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': None},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'lazyLoaderType': {'value': False},
     'enableAdBlockerDetection': {'value': False, 'dropTag': 'server'},
     

In [8]:
if crawler_series['Url'].startswith('https://www.ebay.com/'):
    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
elif crawler_series['Url'].startswith('https://www.ebay.de/'):
    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
elif crawler_series['Url'].startswith('https://www.ebay.co.uk/'):
    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
elif crawler_series['Url'].startswith('https://www.ebay.com.au/'):
    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'
elif crawler_series['Url'].startswith('https://www.ebay.ca/'):
    market, country, language, locale = 'EBAY-ENCA', 'CA', 'en', 'en-CA'

market, country, language, locale

('EBAY-US', 'US', 'en', 'en-US')

In [9]:
dict_['o']['g']['itemId'].strip() != crawler_series['Item Number'] or dict_['o']['g']['marketplaceId'].strip() != market or dict_['o']['g']['country'].strip() != country or dict_['o']['g']['language'].strip() != language or dict_['o']['g']['locale'].strip() != locale

False

In [10]:
list_dict = [list_[2] for list_ in dict_['o']['w'] if 'model' in list_[2]]

list_dict

[{'model': {'lassoExtensions': ['vi_evo'],
   'layout': None,
   'lassoPath': './browser.json',
   'pageRenderSetupCfg': {'templatePathPrefix': 'src/pages/item/',
    'isItmTemplate': True,
    'pageTemplateId': 'ViewItem_Evo'},
   'modules': {'CONDITION_DESCRIPTION': {'_type': 'SectionModule',
     'sections': [{'_type': 'Section',
       'dataItems': [{'_type': 'TextualDisplay',
         'textSpans': [{'_type': 'TextSpan',
           'text': "A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details."}]}]}],
     'title': {'_type': 'TextualDisplay',
      'textSpans': [{'_type': 'TextSpan', 'text': 'New'}]},
     'meta': {'name': 'CONDITION_DESCRIPTION',
      'moduleIdentification': {'instanceId': '1',
       'provider

In [11]:
dict_ = list_dict[0]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in list_dict[0]['model']['modules'] else {}
dict_compatibility = {'page': int(dict_['text'].strip()),
                      'market_place_id': dict_['action']['params']['marketplaceId'],
                      'item_id': dict_['action']['params']['itemId'],
                      'seller_name': dict_['action']['params']['sellerName'],
                      'category_id': dict_['action']['params']['categoryId'],
                      'api': dict_['action']['URL'].strip()} if dict_ else {}

dict_compatibility

{'page': 2,
 'market_place_id': 'EBAY-US',
 'item_id': '232315264904',
 'seller_name': 'scitoo-autoparts',
 'category_id': '33543',
 'api': 'https://www.ebay.com/g/api/finders'}

In [12]:
dict_key, dict_item = {}, list_dict[0]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems'] if 'ABOUT_THIS_ITEM' in list_dict[0]['model']['modules'] else {}
for value in dict_item.values():
    key = value['labels'][0]['textSpans'][0]['text'].strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_specific, dict_count = {}, {}
for i, value in enumerate(dict_item.values()):
    key = value['labels'][0]['textSpans'][0]['text'].strip()
    if dict_key[key] == 1:
        dict_specific[str(i)] = {key: value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
             dict_count[key] = 1
        dict_specific[str(i)] = {f'{key}-{dict_count[key]}': value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

dict_specific

{'0': {'Condition': "New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details."},
 '1': {'Brand': 'SCITOO'},
 '2': {'Manufacturer Warranty': '1 Year'},
 '3': {'UPC': '611040243564'},
 '4': {'Manufacturer Part Number': 'CB104TE056S09'},
 '5': {'Interchange Part Number': 'CO 10845AC'},
 '6': {'Other Part Number': 'MN185237,MR513148, 77495, 6512196, 6512355'},
 '7': {'Superseded Part Number': '78495, 10350931, 1010845, 638912, 77494'},
 '8': {'Compressor type': 'MSC105'},
 '9': {'Voltage': '12V'},
 '10': {'Fitment Type': 'Direct Replacement'},
 '11': {'Type': 'A/C Compressor'},
 '12': {'Kit Parts Included': '1 x Air Conditioner Compressor'},
 '13': {'Oil Type': 'PAG 46'},
 '14': {'Pulley Groove Quantity': '6'},
 '15': 

In [13]:
description_url = list_dict[0]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in list_dict[0]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else list_dict[0]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

description_url

'https://vi.vipr.ebaydesc.com/itmdesc/232315264904?t=1728370349000&category=33543&seller=scitoo-autoparts&excSoj=1&ver=0&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1'

In [14]:
title = list_dict[0]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

title

'AC A/C Compressor For 2004 2005 2006 Mitsubishi Outlander Lancer CO 10845AC 2.4L'

In [15]:
subtitle = list_dict[0]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in list_dict[0]['model']['modules']['TITLE'] else ''

subtitle

''

In [16]:
price = list_dict[0]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip() if 'binModel' in list_dict[0]['model']['modules']['BUY_BOX'] else list_dict[0]['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

price

'US $100.68/ea'

In [17]:
dict_quantity, list_quantity = {}, list_dict[0]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in list_dict[0]['model']['modules'] else []
for i, quantity in enumerate(list_quantity):
    dict_quantity[str(i)] = quantity['text'].strip()

dict_quantity

{'0': 'More than 10 available', '1': '89 sold'}

In [18]:
dict_price, list_dict_price = {}, list_dict[0]['model']['modules']['VOLUME_PRICING']['group']['entries'] if 'VOLUME_PRICING' in list_dict[0]['model']['modules'] else []
for i, dict_ in enumerate(list_dict_price):
    dict_price[str(i)] = dict_['secondaryLabel']['accessibilityText'].strip()

dict_price

{'0': 'Buy 1 for $100.68',
 '1': 'Buy 2 for $98.67 each one',
 '2': 'Buy 3 for $97.66 each one',
 '3': 'Buy 4 for $95.65 each one'}

In [19]:
dict_src, list_media = {}, list_dict[0]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in list_dict[0]['model']['modules']['PICTURE'] else []
for i, media in enumerate(list_media):
    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

dict_src

{'0': 'https://i.ebayimg.com/images/g/404AAOSwR5VisXTT/s-l500.jpg',
 '1': 'https://i.ebayimg.com/images/g/6hEAAOSw4MlisXTU/s-l500.jpg',
 '2': 'https://i.ebayimg.com/images/g/o60AAOSwhtlisXTU/s-l500.jpg',
 '3': 'https://i.ebayimg.com/images/g/9g8AAOSw63lisXTV/s-l500.jpg',
 '4': 'https://i.ebayimg.com/images/g/pWwAAOSw59lisXTV/s-l500.jpg',
 '5': 'https://i.ebayimg.com/images/g/DOcAAOSw8NplLtwK/s-l500.jpg'}

In [20]:
store_name, list_section = '', list_dict[0]['model']['modules']['STORE_INFORMATION']['sections']
for section in list_section:
    if 'logo' in section and 'title' in section['logo']:
        store_name = section['logo']['title']
        break

store_name

'SCITOO-AutoParts'

In [21]:
store_url, list_section = '', list_dict[0]['model']['modules']['STORE_INFORMATION']['sections']
for section in list_section:
    if 'logo' in section and 'action' in section['logo'] and 'URL' in section['logo']['action']:
        store_url = section['logo']['action']['URL'].split('?')[0]
        break

store_url

'https://www.ebay.com/str/cnbatteries'

In [22]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Item Number': crawler_series['Item Number'],
                          'Title': title,
                          'Price': price,
                          'Picture': '',
                          'Url': crawler_series['Url'],
                          'eBay Market': country,
                          'Subtitle': subtitle,
                          'Store Name': store_name,
                          'Store Url': store_url,
                          'Json_Src': json.dumps(dict_src),
                          'Description Url': description_url,
                          'Json_Quantity': json.dumps(dict_quantity),
                          'Json_Price': json.dumps(dict_price),
                          'Json_Specific': json.dumps(dict_specific),
                          'Json_Compatibility': json.dumps(dict_compatibility)})

list_dict_correct

[{'No': 1,
  'Item Number': '232315264904',
  'Title': 'AC A/C Compressor For 2004 2005 2006 Mitsubishi Outlander Lancer CO 10845AC 2.4L',
  'Price': 'US $100.68/ea',
  'Picture': '',
  'Url': 'https://www.ebay.com/itm/232315264904?_ul=US&_stpos=91710&orig_cvip=true',
  'eBay Market': 'US',
  'Subtitle': '',
  'Store Name': 'SCITOO-AutoParts',
  'Store Url': 'https://www.ebay.com/str/cnbatteries',
  'Json_Src': '{"0": "https://i.ebayimg.com/images/g/404AAOSwR5VisXTT/s-l500.jpg", "1": "https://i.ebayimg.com/images/g/6hEAAOSw4MlisXTU/s-l500.jpg", "2": "https://i.ebayimg.com/images/g/o60AAOSwhtlisXTU/s-l500.jpg", "3": "https://i.ebayimg.com/images/g/9g8AAOSw63lisXTV/s-l500.jpg", "4": "https://i.ebayimg.com/images/g/pWwAAOSw59lisXTV/s-l500.jpg", "5": "https://i.ebayimg.com/images/g/DOcAAOSw8NplLtwK/s-l500.jpg"}',
  'Description Url': 'https://vi.vipr.ebaydesc.com/itmdesc/232315264904?t=1728370349000&category=33543&seller=scitoo-autoparts&excSoj=1&ver=0&excTrk=1&lsite=100&ittenable=true&dom

In [23]:
crawler_status = True

crawler_status

True

In [24]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Item Number                                         232315264904
 Url            https://www.ebay.com/itm/232315264904?_ul=US&_...
 Request_Url    https://www.ebay.com/itm/232315264904?_ul=US&_...
 Name: 0, dtype: object]

In [25]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Item Number,Title,Price,Picture,Url,eBay Market,Subtitle,Store Name,Store Url,Json_Src,Description Url,Json_Quantity,Json_Price,Json_Specific,Json_Compatibility
0,1,232315264904,AC A/C Compressor For 2004 2005 2006 Mitsubish...,US $100.68/ea,,https://www.ebay.com/itm/232315264904?_ul=US&_...,US,,SCITOO-AutoParts,https://www.ebay.com/str/cnbatteries,"{""0"": ""https://i.ebayimg.com/images/g/404AAOSw...",https://vi.vipr.ebaydesc.com/itmdesc/232315264...,"{""0"": ""More than 10 available"", ""1"": ""89 sold""}","{""0"": ""Buy 1 for $100.68"", ""1"": ""Buy 2 for $98...","{""0"": {""Condition"": ""New: A brand-new, unused,...","{""page"": 2, ""market_place_id"": ""EBAY-US"", ""ite..."


In [26]:
print('Output Finished !')
print()

Output Finished !



In [27]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Item Number,Url,Request_Url
0,1,232315264904,https://www.ebay.com/itm/232315264904?_ul=US&_...,https://www.ebay.com/itm/232315264904?_ul=US&_...


In [28]:
print('Done ~')

Done ~
